# Importation des librairies

In [ ]:
import os, pandas as pd
import nltk
import numpy
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

# Importation data frames et nettoyage

In [ ]:
DFnames = ["cheyenne", "newYork", "newportBay", "sequoiaLodge", "santaFe", "davyCrockettRanch"]

#Importation from csv
disney = pd.DataFrame(columns = ["Prenom","Note","Pays","Titre","Positif","Négatif","Date séjour","Date commentaire","hotel"])
for i in range(len(DFnames)): 
    nameDF = DFnames[i]
    DF = pd.read_csv("./data/"+nameDF+".csv", index_col=0)  
    DF["hotel"] = nameDF
    disney = pd.concat([disney, DF])
    globals()[nameDF] = DF



In [ ]:
#Analyses
from nettoyage import nettoyage_corpus

#Filtre france en attendant
disney = disney[disney.Pays == "France"]
corpusPos = disney.Positif.tolist()

#nettoyage des nan
corpusPos = [str(i) for i in corpusPos]

#Nettoyage global du corpus
corpusPos = nettoyage_corpus(corpusPos)

from gensim.models import Word2Vec
modele = Word2Vec(corpusPos,vector_size=2,window=3,min_count=1)

#propriété "wv" -> wordvector
words = modele.wv


# Analyse Exploratoire 

In [ ]:
type(modele)

#dimensionnalité
print(modele.vector_size)

In [ ]:
#taille du voisinage
modele.window

#dimension de la représentation
words.vectors.shape


In [ ]:
#affichage des termes de leur index
words.key_to_index

In [ ]:
#taille du dictionnaire
len(words.key_to_index)

In [ ]:
#les clés : les termes
words.key_to_index.keys()

In [ ]:
#similarité entre prix et cher
words.similarity("prix","cher")

In [ ]:
#les termes les plus proches de "prix"
words.most_similar("prix",topn=5)

In [ ]:
#les termes les plus proches de "problème"
words.most_similar("problème",topn=5)

In [ ]:
words.most_similar("personnel",topn=5)

In [ ]:
liste = ['séjour','personnel','prix','bien','super','cher']
words.doesnt_match(liste)

#data frame des coordonnées
import pandas
df = pandas.DataFrame(words.vectors,columns=['V1','V2'],index=words.key_to_index.keys())
print(df)

#sous-data frame corresp. aux termes à étudier
dfListe = df.loc[liste,:]
dfListe

#graphique dans le plan
import matplotlib.pyplot as plt
plt.scatter(dfListe.V1,dfListe.V2,s=0.5)
for i in range(dfListe.shape[0]):
    plt.annotate(dfListe.index[i],(dfListe.V1[i],dfListe.V2[i]))
plt.show()


# Clusters des mots les plus proches

In [ ]:
#fonction pour transformer un document en vecteur
#à partir des tokens qui le composent
#entrée : doc à traiter
#         modèle entrainé ou préentrainé
#sortie : vecteur représentant le document
def my_doc_2_vec(doc,trained):
    #dimension de représentation
    p = trained.vectors.shape[1]
    #initialiser le vecteur
    vec = numpy.zeros(p)
    #nombre de tokens trouvés
    nb = 0
    #traitement de chaque token du document
    for tk in doc:
        #ne traiter que les tokens reconnus
        if ((tk in trained.key_to_index.keys()) == True):
            values = trained[tk]
            vec = vec + values
            nb = nb + 1.0
    #faire la moyenne des valeurs
    #uniquement si on a trouvé des tokens reconnus bien sûr
    if (nb > 0.0):
        vec = vec/nb
    #renvoyer le vecteur
    #si aucun token trouvé, on a un vecteur de valeurs nulles
    return vec


#fonction pour représenter un corpus à partir d'une représentation
#soit entraînée, soit pré-entraînée
#sortie : représentation matricielle
def my_corpora_2_vec(corpora,trained):
    docsVec = list()
    #pour chaque document du corpus nettoyé
    for doc in corpora:
        #calcul de son vecteur
        vec = my_doc_2_vec(doc,trained)
        #ajouter dans la liste
        docsVec.append(vec)
    #transformer en matrice numpy
    matVec = numpy.array(docsVec)
    return matVec

#CAH à partir de scipy
from scipy.cluster.hierarchy import dendrogram, linkage,fcluster

#pour transformation en MDT
from sklearn.feature_extraction.text import CountVectorizer


#fonction pour construire une typologie à partir
#d'une représentation des termes, qu'elle soit entraînée ou pré-entraînée
#seuil par défaut = 1, mais le but est d'avoir 4 groupes
#corpus ici se présente sous la forme d'une liste de listes de tokens
def my_cah_from_doc2vec(corpus,trained,seuil=1.0,nbTermes=7):

    #matrice doc2vec pour la représentation à 100 dim.
    #entraînée via word2vec sur les documents du corpus
    mat = my_corpora_2_vec(corpus,trained)

    #dimension
    #mat.shape

    #générer la matrice des liens
    Z = linkage(mat,method='ward',metric='euclidean')

    #affichage du dendrogramme
    plt.title("CAH")
    dendrogram(Z,orientation='left',color_threshold=0)
    plt.show()

    #affichage du dendrogramme avec le seuil
    plt.title("CAH")
    dendrogram(Z,orientation='left',color_threshold=seuil)
    plt.show()

    #découpage en 4 classes
    grCAH = fcluster(Z,t=seuil,criterion='distance')
    #print(grCAH)

    #comptage
    print(numpy.unique(grCAH,return_counts=True))

    #***************************
    #interprétation des clusters
    #***************************
    
    #parseur
    parseur = CountVectorizer(binary=True)
    
    #former corpus sous forme de liste de chaîne
    corpus_string = [" ".join(doc) for doc in corpus]
    
    #matrice MDT
    mdt = parseur.fit_transform(corpus_string).toarray()
    print("Dim. matrice documents-termes = {}".format(mdt.shape))
    
    #passer en revue les groupes
    for num_cluster in range(numpy.max(grCAH)):
        print("")
        #numéro du cluster à traiter
        print("Numero du cluster = {}".format(num_cluster+1))
        groupe = numpy.where(grCAH==num_cluster+1,1,0)
        effectifs = numpy.unique(groupe,return_counts=True)
        print("Effectifs = {}".format(effectifs[1][1]))
        #calcul de co-occurence
        cooc = numpy.apply_along_axis(func1d=lambda x: numpy.sum(x*groupe),axis=0,arr=mdt)
        #print(cooc)
        #création d'un data frame intermédiaire
        tmpDF = pandas.DataFrame(data=cooc,columns=['freq'],index=parseur.get_feature_names_out())    
        #affichage des "nbTermes" termes les plus fréquents
        print(tmpDF.sort_values(by='freq',ascending=False).iloc[:nbTermes,:])
        
    #renvoyer l'indicateur d'appartenance aux groupes
    return grCAH, mat

#*** fin de la fonction

In [ ]:
#reconstruire la représentation ci-dessus, mais à 100 dim.
modeleBis = Word2Vec(corpusPos,vector_size=100,window=3,min_count=1,epochs=100)
wordsBis = modeleBis.wv

In [ ]:
g1,mat1 = my_cah_from_doc2vec(corpusPos,wordsBis,seuil=10)

# Analyse des topics les plus associés entre eux

In [ ]:
from gensim import corpora

#dictionnaire de mots avec fréquence d'apparition
dictionary = corpora.Dictionary(corpusPos)

# enlever les mots peu utilisés
dictionary.filter_extremes(no_below= 30, keep_n=1000)

#création du Corpus
corpusD = [dictionary.doc2bow(text) for text in corpusPos]

In [ ]:
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpusD, num_topics = 3, id2word=dictionary, passes = 15)

In [ ]:
topics = ldamodel.print_topics(num_words = 4)
print(topics)

#Ajouter des stopwords pour améliorer ? 